
#  <img src="Cython-logo.svg" style="background:none; border:none; box-shadow:none; display:inline; margin:0; vertical-align:middle;" width="25%" height="25%"> 

### A tool for making Python as fast as C/Fortran

# In what way is Python "slow"?

* It is interpreted\*: `python` is a piece of software (written in C) that executes python code on-the-fly. C/Fortran instead are compiled directly to machine-understadable binary code.

<small>* Actually Python is sorta compiled. But the above is still mostly true. </small> 

* In practice this often isn't important because the basic operations are still the same.  But numerical `for` loops (especially with function calls) can be a problem.

* This is why `numpy` was invented - it basically does efficient looping.  But sometimes it's not well suited to a problem...

## Which is clearer?

I want to know the values from `values`, which are aligned to `ids2`, corresponding to the IDs in `ids1`.

In [ ]:
sorti = np.argsort(ids2)
idsforvals = np.searchsorted(ids2[sorti], ids1)
new_array = values[sorti][idsforvals]

# there's actually a better way to do this in newer numpy versions... 
# but it's a contrived example to start with.

In [ ]:
for i in ids1:
    for j in ids2:
        if i == j:
            new_array[i] = values[j]

# What does Cython do about that?

* Cython is an "optimising static compiler" for Python

* It can be run on regular Python... But that usually only has a small effect (not worth the trouble), because the C it makes still does everything `python` does.

* But you can tell it to behave "like C" by specifying `types` for variables.  This can speed things up by orders of magnitude.

* Bonus: you can also call C functions directly

* The price is flexibility (and clarity): you can no longer use Python techniques like freely casting an integer to a float

# Now lets do a "hello world" example

* First make sure you have cython installed (``conda install cython`` should work).
* Look at the file ``hello_world.pyx`` in the spacetelescope/pylunch repo. 
* Open up a terminal and go to the repo and do ``cython hello_world.pyx``.  It should create a ``hello_world.c`` file.
* Now compile that file.  This can be tricky to know the right flags.  For me, this worked:
```
clang -bundle -undefined dynamic_lookup -L/Users/erik/miniconda3/envs/anaconda/lib -arch x86_64 build/temp.macosx-10.6-x86_64-3.5/astropy/_compiler.o -L/Users/erik/miniconda3/envs/anaconda/lib hello_world.c hello_world.so
```
try that but with "anaconda" replaced by your current conda environment.  If you have trouble, check out http://cython.readthedocs.io/en/latest/src/reference/compilation.html .  If you still have trouble, try the method in http://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html, or just wait for a bit.

To actually show this as a presentation, the easiest thing is to do ``jupyter nbconvert --to slides <this file's name>.ipynb --post serve``